In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy import stats
from scipy.stats import kurtosis
from scipy.stats import skew

from io import StringIO # для чтения данных из строки

1. Загрузить файл data_breast.csv. В данном файле собрана расчетная информация с обработанных изображений биоптата молочных желез женщин. Задача заключается в предсказании переменной “Diagnosis” - является ли содержимое биоптата доброкачественным (значение “B” – benign) либо злокачественным (значение “M” – malicious). Описание данных доступно на сайте https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

In [ ]:
data = pd.read_csv('data_breast.csv')

In [ ]:
data.head()

In [ ]:
data.info()

2. Рассчитать основные статистики для переменных (среднее, медиана, мода, мин/макс, сред. отклонение)

In [ ]:
data.describe()

In [ ]:
# Расчитываем основные статистики для средних значений измерений
def describe(df, stats):
    d = df.describe()
    return d.append(df.agg(stats))

describe(data[['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']], ['median'])

3. Выбрать стратегию для работы с пропущенными значениями.

In [ ]:
# проверяем наличие пропущенных значений
data.info()

In [ ]:
#пропущенных значений нет

In [ ]:
data.diagnosis.value_counts()

In [ ]:
# целевая переменная (столбец diagnosis) является категориальной
# переведем значения столбца в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
le.fit( data['diagnosis'] )

In [ ]:
le.classes_

In [ ]:
# записываем в переменную y преобразованный столбец diagnosis
y = pd.Series( data = le.transform( data['diagnosis'] ) )
y.head()

In [ ]:
X = data.drop('diagnosis', axis=1) 
X.head()

In [ ]:
#или меняем значение в датасете

def count_diagnosis(row):
    if 'M' in row['diagnosis']:
        return 1
    else:
        return 0

data['diagnosis'] = data.apply(count_diagnosis, axis=1)
data['diagnosis'].value_counts()


In [ ]:
data.head()

In [ ]:
data["Unnamed: 32"].value_counts(normalize=True, dropna=False)

In [ ]:
#избавляемся от безымянной колонки, заполненной значениями NaN
data.drop(["Unnamed: 32"], axis=1, inplace=True)

4. Рассчитать и визуализировать корреляционную матрицу для переменных.

In [ ]:
corr = data.corr()
corr

In [ ]:
plt.figure(figsize=(25,10))
sns.heatmap(corr, annot=True,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [ ]:
data.rename(columns={'concave points_mean': 'concave_points_mean'}, inplace=True)

In [ ]:
data.rename(columns={'concave points_se': 'concave_points_se'}, inplace=True)

In [ ]:
data.rename(columns={'concave points_worst': 'concave_points_worst'}, inplace=True)

In [ ]:
data.info()

In [ ]:
#оценим корреляцию признаков с target-переменной 'diagnosis'
data.corr()[['diagnosis']].sort_values(by='diagnosis', ascending=False).style.bar()

In [ ]:
#выберем для наглядности ограниченный набор параметров для анализа 
data_mean = data[['diagnosis', 'concave_points_worst', 'perimeter_worst', 'concave_points_mean', 'radius_worst', 'perimeter_mean', 'area_worst', 'radius_mean', 'area_mean', 'concavity_mean', 'concavity_worst', 'compactness_mean', 'compactness_worst', 'radius_se', 'perimeter_se', 'area_se']]


In [ ]:
corr = data_mean.corr()
corr

In [ ]:
plt.figure(figsize=(25,10))
sns.heatmap(data.corr(), annot=True)

5. Визуализировать взаимосвязи между переменными.

In [ ]:
sns.pairplot(data, vars=['diagnosis', 'concave_points_worst', 'perimeter_worst', 'concave_points_mean', 'radius_worst'],
                 kind='scatter')

In [ ]:
sns.pairplot(data, vars=['diagnosis','perimeter_mean', 'area_worst', 'radius_mean', 'area_mean'],
                 kind='scatter')

6. С помощью статистических методов проверить взаимосвязи между переменными.

In [ ]:
stats.ttest_ind(data['concave_points_worst'], data['diagnosis'])

In [ ]:
stats.ttest_rel(data['perimeter_worst'], data['diagnosis'])

In [ ]:
#Проверим взаимосвязь target-переменной c предоставленными признаками в наборе данных

In [ ]:
def get_ols_string(target, array):
    values = " + ".join(np.delete(array, np.argwhere(array==target)))
    return "{0} ~ {1}".format(target, values)

In [ ]:
est = smf.ols(get_ols_string('diagnosis', data.columns.values), data=data).fit()
est_res = est.summary()

In [ ]:
# Функция дл чтения данных из строки
from io import StringIO

In [ ]:
# Изучим результат обучения линейной модели и 
# уберем все переменные первого порядка, для которых верна нулевая гипотеза
table = est_res.tables[1]
pd_table = pd.read_csv(StringIO(table.as_csv()))
pd_table.columns = ['column', 'coef', 'std_err', 't', 'p>t', '0.025', '0.975']

In [ ]:
# Выбираем переменные первого порядка, для которых не верна нулевая гипотеза:
columns = pd_table.sort_values(by='p>t')[(pd_table['p>t']<0.4)].column.unique()
columns = [i.strip(' ') for i in columns]
columns.remove('Intercept')
columns

In [ ]:
# Проверим результаты линейной регрессии на 
est = smf.ols(get_ols_string('diagnosis', columns), data=data).fit()
est_res = est.summary()
est_res.tables[1]

In [ ]:
#est = smf.ols(get_ols_string('diagnosis', data.columns.values), data=data).fit()

est = smf.ols('diagnosis ~ concave_points_worst + perimeter_worst + concave_points_mean + radius_worst', data).fit()
est_res = est.summary()
est_res.tables[1]

In [ ]:
#все выбранные показатели, кроме perimeter_worst, имеют маленький  p-value. 
#Т.е. между perimeter_worst и diagnosis нет линейной зависимости и, следовательно, от этого атрибута можно избавиться.

In [ ]:
three_x_lm = smf.ols('diagnosis ~ area_worst + radius_worst + concave_points_worst', data_mean).fit()
rss = np.sum(three_x_lm.resid ** 2)
print("RSS:", rss)
print("RSE:", np.sqrt(rss / (data.shape[0] - 3 - 1)))
print("R^2:", three_x_lm.rsquared)

In [ ]:
two_x_lm = smf.ols('diagnosis ~ area_worst + radius_worst', data).fit()
rss = np.sum(two_x_lm.resid ** 2)
print("RSS:", rss)
print("RSE:", np.sqrt(rss / (data.shape[0] - 2 - 1)))
print("R^2:", two_x_lm.rsquared)

7. Выбрать стратегию Feature Selection – сокращение размерности либо генерация новых переменных. Какой из этих двух подходов даст лучший результат при классификации?

In [ ]:
#Процедура выбора наиболее значимых атрибутов называется feature selection.

Для того чтобы учитывались не только линейные зависимости при построении модели, расширим количество параметров и проведем для них статистический тест

Для расширения параметров попробуем включить в обучающую выборку нелинейные зависимости, например перемножим все коэффициенты со всеми и изучим результаты статистических тестов, чтобы избавиться от незначительных параметров

In [ ]:
columns_cartesian = [' * '.join([i,j]) for i in columns for j in columns]

# Проверим результаты линейной регрессии
est = smf.ols(get_ols_string('diagnosis', columns+columns_cartesian), data=data).fit()
est_res = est.summary()

table = est_res.tables[1]
pd_table = pd.read_csv(StringIO(table.as_csv()))
pd_table.columns = ['column', 'coef', 'std_err', 't', 'p>t', '0.025', '0.975']
pd_table_filtered = pd_table.sort_values(by='p>t')[(pd_table['p>t']<0.2)]

In [ ]:
# Cобираем итоговую выборку
# Для начала соберем все параметры первого порядка
columns = [i.strip(' ') for i in pd_table_filtered.column.unique()]
columns.remove('Intercept')
parameters = data[[i for i in columns if ':' not in i]]

# Теперь будем перемножать параметры второго порядка
for i in [i for i in columns if ':' in i]:
    i_values = i.split(":")
    parameters[i] = data.apply(lambda x: x[i_values[0]]*x[i_values[1]], axis=1)

In [ ]:
# Подготовка данных для решения задачи
X = parameters
y = data['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.33, random_state=42)

8. Провести стратегию Oversampling/Undersampling, проверить дает ли она улучшение результатов.

In [ ]:
#посмотрим на распределение значений для target-переменной 'diagnosis'
plt.title('Распределение положительных и отрицательных результатов тестов')
plt.hist(y)

plt.hist(y_train)

In [ ]:
#количество доброкачественных результатов тестов и злокачественных различается, что может повлиять на переобучение модели. 
#при этом данных в исходной выборке не очень много, поэтому попробуем применить стратегию Oversampling
#импортируем библиотеку для применения стратегии OVERSAMPLING
from imblearn.over_sampling import SMOTE

In [ ]:
X_resampled_o, y_resampled_o = SMOTE().fit_resample(X, y)
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X_resampled_o, y_resampled_o,  test_size=0.33, random_state=42)

In [ ]:
plt.title('Распределение положительных и отрицательных результатов тестов')
plt.hist(y_resampled_o)

plt.hist(y_train_o)#после применения стратегии OVERSAMPLING

9. Сделать кросс-валидацию данных с использованием подхода K-fold (n_folds=10).

In [ ]:
model_logreg = LogisticRegression(penalty='l1', C=0.1, solver='liblinear')
model_logreg_o = LogisticRegression(penalty='l1', C=0.1, solver='liblinear')

In [ ]:
# Оценка модели на кросс-валидации и оверсэмплинге
cross_val_score(model_logreg_o, X_train_o, y_train_o, cv=10).mean()

In [ ]:
# Оценка модели на кросс-валидации без оверсэмплинга
cross_val_score(model_logreg, X_train, y_train, cv=10).mean()

In [ ]:
model_logreg_o.fit(X_train_o, y_train_o)
model_logreg.fit(X_train, y_train)

In [ ]:
# Оценка модели на данных с оверсемплингом
model_logreg_o.score(X_test, y_test)

In [ ]:
# Оценка модели на данных без оверсемплинга
model_logreg.score(X_test, y_test)

In [ ]:
logreg_prediction = model_logreg_o.predict(X_test)
logreg_prediction_proba = model_logreg_o.predict_proba(X_test)

In [ ]:
#овэрсэмплинг для данной задачи дает значительный прирост к точности модели

10. Рассчитать Feature Selection для выбранных переменных.

In [ ]:
#Выбрать те переменные которые по вашему мнению оптимальны для получения лучших результатов при тренировке модели

In [ ]:
#сделано на шаге 7

11. Решить задачу бинарной классификации и предсказать переменную ”Diagnosis ” протестировав как минимум 2
алгоритма. Использовать те алгоритмы, которые позволяют предсказать вероятность класса (proba). Рассчитать и
вывести вероятность каждого класса.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit( X, y )
predictions = model.predict_proba( X )

In [ ]:
predictions[:5]

In [ ]:
model.score(X_train, y_train)

12. Проверить качество классификации с использованием следующих метрик: Accuracy, F1-Score, Precision, Recall

### Accuracy
$$ Accuracy=P/N $$
где, P – количество документов по которым классификатор принял правильное решение, а N – размер обучающей выборки.

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
clf = LogisticRegression(C=best_C)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_test_pred = clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_test_pred)
#или
#accuracy_score(y_test, logreg_prediction)

### F1-Score
Понятно что чем выше точность и полнота, тем лучше. Но в реальной жизни максимальная точность и полнота не достижимы одновременно и приходится искать некий баланс. Поэтому, хотелось бы иметь некую метрику которая объединяла бы в себе информацию о точности и полноте нашего алгоритма.

F-мера представляет собой гармоническое среднее между точностью и полнотой. Она стремится к нулю, если точность или полнота стремится к нулю.

$$ F=2\dfrac{Precision×Recall}{Precision+Recall} $$

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, logreg_prediction)

### Precision-Recall

TP — истино-положительное решение;
TN — истино-отрицательное решение;
FP — ложно-положительное решение;
FN — ложно-отрицательное решение. Тогда, точность и полнота определяются следующим образом:
$$ Precision=TP/(TP+FP) $$$$ Recall=TP/(TP+FN) $$
Precision - количество истинно положительных результатов от количества модельно-положительных результатов (фактически точность модели) Recall - количество истинно положительных относительно реально-положительных результатов (фактически отклик на реальность)

In [ ]:
from sklearn.metrics import precision_recall_curve, precision_score, recall_score

In [ ]:
print(" Precision = {0}; Recall = {1}".\
    format(precision_score(y_test, logreg_prediction), recall_score(y_test, logreg_prediction)))

In [ ]:
# Построим кривую Precision-Recall для Логистической регрессии
precision, recall, thresholds = precision_recall_curve(y_test, 
                                    [i[1] for i in logreg_prediction_proba])

plt.figure(figsize=(8,8))
plt.title('Precision-Recall Curve for Logistic Regression')
plt.rcParams.update({'font.size': 22})
plt.xlabel("Recall")
plt.ylabel("Precision")
sns.lineplot(recall, precision)

### Confusion matrix (Матрица неточностей)
Столбцы этой матрицы резервируются за экспертными решениями, а строки за решениями классификатора.

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# Confusion matrix для линейной регрессии
confusion_matrix(y_test, logreg_prediction)

13. Проверить качество вероятности класса с использованием метрики: Brier Score

The most common formulation of the Brier score is

$$ BS={\frac {1}{N}}\sum \limits _{t=1}^{N}(f_{t}-o_{t})^{2} $$
in which  $f_{t}$ is the probability that was forecast,  $o_{t}$ the actual outcome of the event at instance t (0 if it does not happen and 1 if it does happen) and N is the number of forecasting instances.

In [ ]:
from sklearn.metrics import brier_score_loss

In [ ]:
brier_score_loss(y_test, [i[1] for i in logreg_prediction_proba])